# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-06-25 03:18:46] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=36742, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=744035180, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, ba

[2025-06-25 03:18:53] Attention backend not set. Use fa3 backend by default.
[2025-06-25 03:18:53] Init torch distributed begin.


[2025-06-25 03:18:53] Init torch distributed ends. mem usage=0.00 GB


[2025-06-25 03:18:54] Load weight begin. avail mem=61.98 GB


[2025-06-25 03:18:54] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-06-25 03:18:58] INFO:     Started server process [192381]
[2025-06-25 03:18:58] INFO:     Waiting for application startup.
[2025-06-25 03:18:58] INFO:     Application startup complete.
[2025-06-25 03:18:58] INFO:     Uvicorn running on http://0.0.0.0:36742 (Press CTRL+C to quit)


[2025-06-25 03:18:59] INFO:     127.0.0.1:53632 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-25 03:18:59] INFO:     127.0.0.1:53638 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-25 03:18:59] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 03:19:00] INFO:     127.0.0.1:53644 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 03:19:00] The server is fired up and ready to roll!


Server started on http://localhost:36742


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-06-25 03:19:04] INFO:     127.0.0.1:45734 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-06-25 03:19:04] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 03:19:04] INFO:     127.0.0.1:45738 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-06-25 03:19:04] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-25 03:19:04] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, cuda graph: False, gen throughput (token/s): 5.99, #queue-req: 0


[2025-06-25 03:19:05] INFO:     127.0.0.1:45752 - "POST /generate HTTP/1.1" 200 OK


[2025-06-25 03:19:05] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 73, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-25 03:19:05] Decode batch. #running-req: 1, #token: 101, token usage: 0.00, cuda graph: False, gen throughput (token/s): 66.83, #queue-req: 0


[2025-06-25 03:19:06] Decode batch. #running-req: 1, #token: 141, token usage: 0.01, cuda graph: False, gen throughput (token/s): 68.82, #queue-req: 0


[2025-06-25 03:19:06] INFO:     127.0.0.1:45766 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-06-25 03:19:06] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-25 03:19:06] INFO:     127.0.0.1:45780 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 03:19:06] Prefill batch. #new-seq: 2, #new-token: 5, #cached-token: 62, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-25 03:19:06] INFO:     127.0.0.1:45788 - "POST /generate HTTP/1.1" 200 OK


[2025-06-25 03:19:06] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 03:19:06] Decode batch. #running-req: 1, #token: 68, token usage: 0.00, cuda graph: False, gen throughput (token/s): 53.31, #queue-req: 0


[2025-06-25 03:19:07] INFO:     127.0.0.1:45800 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-06-25 03:19:07] Prefill batch. #new-seq: 2, #new-token: 70, #cached-token: 28, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 03:19:07] Decode batch. #running-req: 2, #token: 98, token usage: 0.00, cuda graph: False, gen throughput (token/s): 96.78, #queue-req: 0


[2025-06-25 03:19:08] Decode batch. #running-req: 2, #token: 178, token usage: 0.01, cuda graph: False, gen throughput (token/s): 135.41, #queue-req: 0


[2025-06-25 03:19:08] Decode batch. #running-req: 2, #token: 258, token usage: 0.01, cuda graph: False, gen throughput (token/s): 135.25, #queue-req: 0


[2025-06-25 03:19:09] Decode batch. #running-req: 2, #token: 338, token usage: 0.02, cuda graph: False, gen throughput (token/s): 135.38, #queue-req: 0


...

[2025-06-25 03:19:09] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, cuda graph: False, gen throughput (token/s): 110.62, #queue-req: 0
[2025-06-25 03:19:09] INFO:     127.0.0.1:45804 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 03:19:09] Prefill batch. #new-seq: 1, #new-token: 376, #cached-token: 39, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 03:19:10] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, cuda graph: False, gen throughput (token/s): 62.06, #queue-req: 0


[2025-06-25 03:19:11] Decode batch. #running-req: 1, #token: 495, token usage: 0.02, cuda graph: False, gen throughput (token/s): 67.08, #queue-req: 0


[2025-06-25 03:19:11] Decode batch. #running-req: 1, #token: 535, token usage: 0.03, cuda graph: False, gen throughput (token/s): 67.04, #queue-req: 0


[2025-06-25 03:19:11] INFO:     127.0.0.1:45814 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-06-25 03:19:11] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 03:19:13] INFO:     127.0.0.1:45830 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-06-25 03:19:13] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-25 03:19:14] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, cuda graph: False, gen throughput (token/s): 16.98, #queue-req: 0


[2025-06-25 03:19:14] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, cuda graph: False, gen throughput (token/s): 68.53, #queue-req: 0


[2025-06-25 03:19:15] Decode batch. #running-req: 1, #token: 126, token usage: 0.01, cuda graph: False, gen throughput (token/s): 67.08, #queue-req: 0


[2025-06-25 03:19:15] INFO:     127.0.0.1:45844 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-06-25 03:19:15] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-25 03:19:15] INFO:     127.0.0.1:34028 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:00,  6.88it/s]

 67%|██████▋   | 2/3 [00:00<00:00, 10.91it/s]

[2025-06-25 03:19:15] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, cuda graph: False, gen throughput (token/s): 65.18, #queue-req: 0
[2025-06-25 03:19:15] Prefill batch. #new-seq: 3, #new-token: 29, #cached-token: 51, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-25 03:19:15] INFO:     127.0.0.1:34058 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 03:19:15] INFO:     127.0.0.1:34044 - "POST /generate HTTP/1.1" 200 OK


100%|██████████| 3/3 [00:00<00:00, 17.17it/s]

[2025-06-25 03:19:15] INFO:     127.0.0.1:34040 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-06-25 03:19:15] INFO:     127.0.0.1:34062 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 03:19:15] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0


The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-06-25 03:19:16] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 03:19:16] INFO:     127.0.0.1:34072 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-06-25 03:19:16] Child process unexpectedly failed with exitcode=9. pid=192921


## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-06-25 03:19:22] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=37472, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=893400371, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-06-25 03:19:24] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-06-25 03:19:24] Inferred chat template from model path: qwen2-vl


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-06-25 03:19:30] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-06-25 03:19:30] Attention backend not set. Use flashinfer backend by default.
[2025-06-25 03:19:30] Automatically reduce --mem-fraction-static to 0.787 because this is a multimodal model.
[2025-06-25 03:19:30] Init torch distributed begin.


...

[2025-06-25 03:19:38] INFO:     Started server process [193959]
[2025-06-25 03:19:38] INFO:     Waiting for application startup.
[2025-06-25 03:19:38] INFO:     Application startup complete.
[2025-06-25 03:19:38] INFO:     Uvicorn running on http://0.0.0.0:37472 (Press CTRL+C to quit)


[2025-06-25 03:19:39] INFO:     127.0.0.1:41174 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-25 03:19:39] INFO:     127.0.0.1:41186 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-25 03:19:39] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 03:19:40] INFO:     127.0.0.1:41194 - "POST /generate HTTP/1.1" 200 OK
[2025-06-25 03:19:40] The server is fired up and ready to roll!


Server started on http://localhost:37472


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-06-25 03:19:44] INFO:     127.0.0.1:49104 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-06-25 03:19:45] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-25 03:19:46] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, cuda graph: False, gen throughput (token/s): 4.59, #queue-req: 0


[2025-06-25 03:19:47] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, cuda graph: False, gen throughput (token/s): 40.84, #queue-req: 0


[2025-06-25 03:19:48] INFO:     127.0.0.1:49108 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)

[2025-06-25 03:19:48] Child process unexpectedly failed with exitcode=9. pid=194171
